# 14.6 子词嵌入(Subword Embedding)
- **目录**
  - 14.6.1 fastText模型
  - 14.6.2 字节对编码

- 在英语中，“helps”、“helped”和“helping”等单词都是同一个词“help”的变形形式。
- “dog”和“dogs”之间的关系与“cat”和“cats”之间的关系相同，“boy”和“boyfriend”之间的关系与“girl”和“girlfriend”之间的关系相同。
- 在法语和西班牙语等其他语言中，许多动词有40多种变形形式，而在芬兰语中，名词最多可能有15种变形。
- 在语言学中，**形态学研究单词形成和词汇关系**。
- 但是，word2vec和GloVe都没有对词的内部结构进行探讨。

## 14.6.1 fastText模型

回想一下词在word2vec中是如何表示的。在跳元模型和连续词袋模型中，**同一词的不同变形形式直接由不同的向量表示**，不需要共享参数。为了使用形态信息，**fastText模型**提出了一种**子词嵌入**方法，其中子词是一个字符$n$-gram 。fastText可以被认为是**子词级跳元模型**，而非学习词级向量表示，其中每个**中心词**由其**子词级向量之和**表示。

让我们来说明如何以单词“where”为例获得fastText中每个中心词的子词。首先，在词的开头和末尾添加特殊字符“&lt;”和“&gt;”，以将前缀和后缀与其他子词区分开来。
然后，从词中提取字符$n$-gram。
例如，值$n=3$时，我们将获得长度为3的所有子词：
“&lt;wh”、“whe”、“her”、“ere”、“re&gt;”和特殊子词“&lt;where&gt;”。

在fastText中，对于任意词$w$，用$\mathcal{G}_w$表示其长度在3和6之间的所有子词与其特殊子词的并集。词表是所有词的子词的集合。假设$\mathbf{z}_g$是词典中的子词$g$的向量，则跳元模型中作为中心词的词$w$的向量$\mathbf{v}_w$是其子词向量的和：

$$\mathbf{v}_w = \sum_{g\in\mathcal{G}_w} \mathbf{z}_g. \tag{14.6.1}$$

fastText的其余部分与跳元模型相同。与跳元模型相比，fastText的词量更大，模型参数也更多。此外，**为了计算一个词的表示，它的所有子词向量都必须求和**，这导致了更高的计算复杂度。然而，**由于具有相似结构的词之间共享来自子词的参数，罕见词甚至词表外的词在fastText中可能获得更好的向量表示**。

In [1]:
# fastText的n-gram生成
word = "where"
word = '<' + word + '>'
ngrams = []
for n in range(3, 7):
    for i in range(len(word) - n + 1):
        ngrams.append(word[i:i+n])
ngrams.append(word)
print("FastText子词:", ngrams)

FastText子词: ['<wh', 'whe', 'her', 'ere', 're>', '<whe', 'wher', 'here', 'ere>', '<wher', 'where', 'here>', '<where', 'where>', '<where>']


- **要点：**
  - **词的表示**：在word2vec中，同一词的不同形态变形是通过不同的向量直接表示的，不共享参数。
  - **子词嵌入**：为了利用**形态信息**，fastText模型引入了子词嵌入方法。这里的子词指的是字符$n$-gram。
  - **中心词表示**：fastText可以看作是子词级的跳元模型。与其学习词级的向量表示，每个中心词在fastText中是由其子词的向量之和表示的。
  - **子词的提取**：
    - 为词加上特殊字符“<”和“>”标记词的开头和结尾。
    - 提取长度为$n$的字符子词。例如，对于词“where”且$n=3$，我们有子词：“<wh”、“whe”、“her”、“ere”、“re>”和特殊子词“<where>”。
  - **子词集合**：对于任何词$w$，$\mathcal{G}_w$表示其所有长度在3和6之间的子词加上它的特殊子词的集合。
  - **词表**：是所有词的子词的集合。词$w$的向量表示$\mathbf{v}_w$是其所有子词向量的和，如公式\(14.6.1\)所示。
  - **与跳元模型的对比**：
    - fastText的词量更大，有更多的模型参数。
    - 计算一个词的表示需要求所有子词向量的和，导致更高的计算复杂度。
    - 但由于形态相似的词可以共享子词参数，所以罕见词和词表外的词在fastText中可能获得更好的向量表示。

## 14.6.2 字节对编码（Byte Pair Encoding）
- 在fastText中，所有提取的子词都必须是指定的长度，例如$3$到$6$，因此词表大小不能预定义。
- 为了在固定大小的词表中**允许可变长度的子词**，可以应用一种称为**字节对编码（Byte Pair Encoding，BPE）** 的压缩算法来提取子词。
  - 字节对编码执行训练数据集的统计分析，以发现单词内的**公共符号**，诸如任意长度的连续字符。
  - 从长度为1的符号开始，字节对编码**迭代地合并最频繁的连续符号对**以产生新的更长的符号。
    - 请注意，为提高效率，不考虑跨越单词边界的对。
  - 最后可以使用像**子词**这样的符号来切分单词。
  - 字节对编码及其变体已经用于诸如GPT-2和RoBERTa等自然语言处理预训练模型中的输入表示。
- GPT-3也使用BPE算法构建词表：
  - 词表包含50,257个token，包含常见单词、子词（subword）、符号和部分多语言字符。
  - 其中256个token 保留给特殊控制字符（如换行符、制表符等）。
  - 中文处理：
    - 单个汉字通常作为独立 token（如 "人工智能" → "人" + "工" + "智" + "能"），效率较低。
    - 部分常见词组可能合并（如 “北京” → 单个 token）。
  - 代码处理：编程语言关键词（如 if, def）和符号（如 +=）有独立 token。
  - GPT-3通过BPE编码的语料库大概有400B(4000亿)个左右的token，处理之前的原始语料是45T。

  

- 字节对编码算法包含如下步骤及其变量与函数：
  - 初始化符号词表symbols。
  - 统计训练数据中每个词及其频率raw_token_freqs。
  - 将每个词拆分为单字符并用空格分隔，形成符号序列token_freqs。
  - 迭代合并高频符号对：
    - 使用get_max_freq_pair函数统计相邻符号对频率。
    - 使用merge_symbols函数合并最高频符号对
  - 生成最终子词词表。
  - 使用segment_BPE函数进行语料数据的子词分割（推理阶段）。

- 首先，此处将符号词表初始化为所有**英文小写字符**、**特殊的词尾函数符号`'_'`** 和 **特殊的未知符号`'[UNK]'`**。

In [2]:
import collections

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

- 因为不考虑跨越词边界的符号对（即仅统计和合并同一个单词内部的连续字符对），所以我们只需要一个字典`raw_token_freqs`将词映射到数据集中的频率（出现次数）。
- 注意，特殊符号`'_'`被附加到每个词的尾部，以便可以容易地从输出符号序列（例如，“a_ tall er_ man”）恢复单词序列（例如，“a taller man”）。
- 由于我们仅从单个字符和特殊符号的词开始合并处理，所以在每个词（词典`token_freqs`的键）内的每对连续字符之间插入空格。
- 换句话说，**空格是词中符号之间的分隔符**。


In [3]:
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}

for token, freq in raw_token_freqs.items():
    # 注意list和' '.join的用法
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

- 定义以下`get_max_freq_pair`函数：
  - 该函数返回词内**最频繁的连续符号对**。
  - 其中词来自输入词典`token_freqs`的键。


In [4]:
## 原代码
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            pairs[symbols[i], symbols[i + 1]] += freq
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

- get_max_freq_pair完整注释

In [14]:
## 获取最频繁的符号对
## 接受一个词频词典token_freqs作为参数
def get_max_freq_pair(token_freqs):
    '''
    使用collections.defaultdict初始化一个默认字典pairs。
    该字典用于存储符号对及其出现的频率。
    defaultdict(int)意味着任何尚未存在于字典中的键都会默认具有值int()，也就是0。
    '''
    pairs = collections.defaultdict(int)
    ## 开始遍历token_freqs字典的每个条目，其中token是词，freq是词的出现频率
    for token, freq in token_freqs.items():
        ## 将token（一个由空格分隔的符号字符串）拆分成一个符号列表，并赋值给symbols
        symbols = token.split()
        ## 遍历symbols列表以便查找两个连续符号，但停在倒数第二个符号
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            '''
            更新pairs字典中当前连续符号对的频率。
            键是符号对的元组(symbols[i], symbols[i + 1])，增加该键对应的频率值freq，
            比如'f a'在'f a s t _'的词频是4, 在'f a s t e r _'是3，二者相加等于7。
            '''
            pairs[symbols[i], symbols[i + 1]] += freq
    
    '''
    在所有连续符号对中返回具有最大频率的那对。
    key=pairs.get意味着使用pairs词典的值（即频率）作为决定“最大”的标准。
    '''
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

In [5]:
## 获取最频繁符号对方法调用示例
get_max_freq_pair(token_freqs)

('t', 'a')

In [6]:
# max与字典类型结合的用法
d = {('a','b'):10, ('b','c'):20, ('c','d'):30}
max(d, key=d.get)

('c', 'd')

- 作为基于连续符号频率的贪心方法，字节对编码将使用以下`merge_symbols`函数来合并最频繁的连续符号对以产生新符号。


In [7]:
## 原代码
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

- merge_symbols完整注释

In [18]:
## 注释代码
'''
两种解释：
（1）合并最高频率的符号对，并更新词频字典来反映这种合并。
     这是字节对编码（BPE）算法中的一个关键步骤，
     持续合并最常见的符号对，直到达到所需的词表大小或其他停止条件。
（2）将token_freqs中出现频率最高的符号对max_freq_pair合并，
     并更新token_freqs字典来反映这种合并。
     同时，新的合并符号被添加到symbols列表中。
'''
## 三个参数：最高频率的符号对max_freq_pair、词频字典token_freqs和符号列表symbols。
def merge_symbols(max_freq_pair, token_freqs, symbols):
    '''
    将max_freq_pair中的两个符号合并成一个字符串，
    然后将其添加到symbols列表的末尾。
    例如，如果max_freq_pair是('t', 'a')，
    则它会被合并为'ta'并添加到symbols列表中。
    '''
    symbols.append(''.join(max_freq_pair))
    ## 新字典对象用于存储合并后的词及其频率
    new_token_freqs = dict()
    
    for token, freq in token_freqs.items():
        '''
        替换词中的最高频率的符号对。
        它首先将max_freq_pair中的符号转换为由空格分隔的字符串（例如，'t a'），
        然后在token中找到这个字符串并将其替换为无空格的版本（例如，'ta'）。
        结果存储在new_token中。
        '''
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        ## 将原始频率从token_freqs复制到新词典new_token_freqs，以new_token为键。
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

In [8]:
## merge_symbols调用示例
w = get_max_freq_pair(token_freqs)
merge_symbols(w, token_freqs, symbols)

{'f a s t _': 4, 'f a s t e r _': 3, 'ta l l _': 5, 'ta l l e r _': 4}

- 现在对词典`token_freqs`的键**迭代地**执行字节对编码算法。
  - 在第一次迭代中，最频繁的连续符号对是`'t'`和`'a'`，因此字节对编码将它们合并以产生新符号`'ta'`。
  - 在第二次迭代中，字节对编码继续合并`'ta'`和`'l'`以产生另一个新符号`'tal'`。


In [9]:
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'合并# {i+1}:',max_freq_pair)

合并# 1: ('t', 'a')
合并# 2: ('ta', 'l')
合并# 3: ('tal', 'l')
合并# 4: ('f', 'a')
合并# 5: ('fa', 's')
合并# 6: ('fas', 't')
合并# 7: ('e', 'r')
合并# 8: ('er', '_')
合并# 9: ('tall', '_')
合并# 10: ('fast', '_')


- 在字节对编码的10次迭代之后，可以看到列表`symbols`现在又包含10个从其他符号迭代合并而来的符号。


In [10]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


- 对于在词典`raw_token_freqs`的键中指定的同一数据集，作为字节对编码算法的结果，数据集中的每个词现在被子词“fast_”、“fast”、“er_”、“tall_”和“tall”分割。
  - 例如，单词“faster_”和“taller_”分别被分割为“fast er_”和“tall er_”。


In [11]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']


In [12]:
token_freqs

{'fast_': 4, 'fast er_': 3, 'tall_': 5, 'tall er_': 4}

- 请注意，字节对编码的结果取决于正在使用的数据集。
- 还可以使用**从一个数据集学习的子词来切分另一个数据集的单词**。
- 作为一种**贪心方法**，下面的`segment_BPE`函数尝试将单词从输入参数`symbols`分成**可能最长**的子词。


In [13]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # 具有符号中可能最长子字的词元段
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

- segment_BPE详细注释

In [14]:
## tokens是待分割的新词元，一般是完整的词元加上"_"符号
## symbols是经过BPE子词划分后的符号，是子词分割的依据
def segment_BPE(tokens, symbols):
    outputs = [] #保存子词分割后的列表
    for token in tokens: #取出一个词元
        start, end = 0, len(token) #子词分割的起始点和结束点
        cur_output = [] #当前词元的子词划分
        # 具有符号中可能最长子字的词元段
        '''
        对于词元的子词分割使用贪婪算法，即从最大长度开始匹配。
        （1）如果最大长度的子词在符号列表symbols中存在，
             即将其作为子词存放在当前词元的子词列表cur_output中。
             然后将start赋值为end。
        （2）如果token[start: end]在symbols不存在，则将end减1，
             此时token[start: end]的字符减少一个即最后一个字符。
             然后再将token[start: end]匹配symbols，即回到步骤（1）。
             
        以token='tallest_'为例：
        （1）按照贪心算法多次迭代即end-1后，
             symbols中的'tall'匹配。
        （2）然后start=end，此时start指向'tallest_'
             中的'e'，end=len(token)再次指向'tallest_'的末尾后面一个字符。
        （3）对剩下部分"est_"进行子词分割,回到第一步。

        注意end是词元长度，按照列表的索引方式，它是指向token最后一个字符的后面一个单元。
        比如刚开始迭代时，token[start: end]包含整个token的所有字符。
        '''
        while start < len(token) and start < end: #示例：tallest_
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        # 如果直到匹配结束在symbols仍找不到子词，则将'[UNK]'添加到当前词元的子词列表中
        # 表示未知符号或子词
        if start < len(token):
            cur_output.append('[UNK]')
        # 将当前词元的子词使用空格分开，然后作为一个完整字符串添加到最终输出
        outputs.append(' '.join(cur_output))
    return outputs

- 使用列表`symbols`中的子词（从前面提到的数据集学习）来表示另一个数据集的`tokens`。


In [14]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, symbols))

['tall e s t _', 'fa t t er_']


## 小结

* fastText模型提出了一种子词嵌入方法：基于word2vec中的跳元模型，它将中心词表示为其子词向量之和。
* 字节对编码执行训练数据集的统计分析，以发现词内的公共符号。作为一种贪心方法，字节对编码迭代地合并最频繁的连续符号对。
* 子词嵌入可以提高稀有词和词典外词的表示质量。

-------
- **说明：DeepSeek的词表构建与分词技术**
  - DeepSeek 作为一款强大的中文大模型，在词表（Tokenization）设计上采用了多项针对中文优化的技术，以解决传统BPE方法在中文处理上的效率低下问题。
  - **1. 混合分词策略（Hybrid Tokenization）**
    - **问题**：传统 BPE 对中文按单字拆分（如 "人工智能" → `"人"+"工"+"智"+"能"`），导致 token 数量爆炸，效率低下。  
    - **DeepSeek 的解决方案：结合 BPE 与词典分词**：  
      - 预加载高频中文词汇（如“人工智能”“北京”），直接保留为完整 token，减少拆分。  
      - 低频词仍按 BPE 子词规则处理。  
    - **示例**：  
      - 传统 BPE："人工智能" → 4 tokens  
      - DeepSeek："人工智能" → 1 token（若在预置词典中）
  - **2. 汉字优先编码（Chinese-Character-Centric BPE）**
    - **问题**：通用 BPE 对多语言混合优化，中文压缩率低。  
    - **DeepSeek 的优化：单独训练中文子词表**：  
      - 在中文语料上独立训练 BPE，优先合并常见汉字组合（如“的”“是”“中国”）。  
      - 英文和符号沿用标准 BPE，但权重降低。  
    - **效果**：  
      - 中文平均 token 数减少 30%-50%（如句子“深度学习很重要”从 7 tokens 降至 3-4 tokens）。
  - **3. 动态分词（Dynamic Tokenization）**
    - **问题**：专业领域术语（如“Transformer”“贝叶斯”）可能被错误拆分。  
    - **DeepSeek 的改进：领域自适应词表**  
      - 针对医学、法律、编程等领域，动态加载领域词典，强制保留术语为完整 token。  
        - 例如：“冠状动脉”在医疗文本中作为 1 个 token，而非 `"冠状"+"动脉"`。  
      - **用户自定义词表**：允许用户添加新词（如品牌名“深度求索”），避免拆分。
  - **4. Unicode 归一化与繁体字处理**
    - **问题**：中文存在简繁变体（如“语” vs “語”），增加词表冗余。  
    - **DeepSeek 的优化**：
      - **简繁映射**：  
        - 在 tokenizer 预处理阶段，将繁体字自动转换为简体（可配置关闭）。  
        - 减少词表中重复 token（如“说”和“說”合并）。  
      - **全角/半角统一**： 将全角符号$，．$转为半角$, .$，降低符号多样性。
  - **5. 高频子词强制合并**
    - **问题**：常见中文后缀（如“们”“的”“性”）被重复编码。  
    - **DeepSeek 的策略：统计驱动合并**
      - 强制合并高频后缀/前缀（如“的”、“了”、“主义”），即使 BPE 统计频率未达阈值。  
      - 例如：“科学家们” → `"科学家" + "们"`（而非 `"科学" + "家" + "们"`）。

  - **6. 与其他语言的兼容性**
    - **多语言混合处理**：  
      - **分层词表**：  
        - 中文和英文分别使用独立的子词表，在模型输入层拼接。  
        - 避免中英混合文本的冲突（如“ChatGPT”和“聊天机器人”各自优化）。  
      - **代码处理**：保留编程语言关键词（如 `if`, `def`）为完整 token，与中文自然分词隔离。


- **附录：fastText模型训练代码示例**

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, Counter
import random
import numpy as np
from tqdm import tqdm

def get_ngrams(word, min_n=3, max_n=6):
    word = "<" + word + ">"
    ngrams = []
    for n in range(min_n, max_n + 1):
        for i in range(len(word) - n + 1):
            ngrams.append(word[i:i+n])
    return ngrams + ["<" + word[1:-1] + ">"]  # 添加整个单词作为特殊子词

def build_vocab(corpus, min_count=1):
    word_counts = Counter(corpus)
    vocab = {word: idx for idx, (word, count) in enumerate(word_counts.items())}
    
    # 为所有单词生成子词并加入词汇表
    subword_vocab = set()
    for word in vocab:
        subword_vocab.update(get_ngrams(word))
    
    # 为子词分配索引，从len(vocab)开始
    subword_vocab = {g: idx + len(vocab) for idx, g in enumerate(subword_vocab)}
    
    # 合并单词和子词词汇表
    vocab.update(subword_vocab)
    return vocab

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(FastText, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.subword_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

    def forward(self, word_ids, subword_ids_list):
        word_vecs = self.word_embeddings(word_ids)
        
        subword_vecs = []
        for subword_ids in subword_ids_list:
            # 确保所有子词索引有效
            valid_subword_ids = [idx for idx in subword_ids if idx < self.vocab_size]
            if not valid_subword_ids:
                # 如果没有有效子词，使用零向量
                subword_vec = torch.zeros(self.embedding_dim)
            else:
                subword_vec = self.subword_embeddings(torch.LongTensor(valid_subword_ids))
                subword_vec = torch.sum(subword_vec, dim=0)
            subword_vecs.append(subword_vec)
        
        subword_vecs = torch.stack(subword_vecs)
        combined_vecs = word_vecs + subword_vecs
        return combined_vecs

def negative_sampling(target_word_idx, vocab, num_neg_samples=5):
    valid_indices = [idx for idx in range(len(vocab)) if idx != target_word_idx]
    return random.sample(valid_indices, min(num_neg_samples, len(valid_indices)))

def train_fasttext(corpus, embedding_dim=100, epochs=10, lr=0.01):
    vocab = build_vocab(corpus)
    train_data = []
    
    # 构建训练数据 (中心词, 上下文词) 对
    window_size = 2
    for i, center_word in enumerate(corpus):
        for j in range(max(0, i-window_size), min(len(corpus), i+window_size+1)):
            if i != j:
                train_data.append((center_word, corpus[j]))
    
    model = FastText(len(vocab), embedding_dim)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        total_loss = 0
        for center_word, context_word in tqdm(train_data):
            # 确保单词在词汇表中
            if center_word not in vocab or context_word not in vocab:
                continue
                
            center_id = vocab[center_word]
            context_id = vocab[context_word]
            
            # 获取中心词的子词
            subword_ids = [vocab[g] for g in get_ngrams(center_word) if g in vocab]
            
            # 前向传播
            combined_vec = model(torch.LongTensor([center_id]), [subword_ids])
            
            # 负采样
            neg_samples = negative_sampling(context_id, vocab)
            neg_samples_vec = model.word_embeddings(torch.LongTensor(neg_samples))
            
            # 计算损失
            pos_score = torch.matmul(combined_vec, model.word_embeddings.weight[context_id].unsqueeze(0).T)
            neg_scores = torch.matmul(combined_vec, neg_samples_vec.T)
            logits = torch.cat([pos_score, neg_scores], dim=1)
            labels = torch.LongTensor([0])  # 正样本在logits的第0位置
            loss = criterion(logits, labels)
            
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_data)}")

    return model, vocab

# 训练
corpus = ["I", "love", "natural", "language", "processing", "fasttext", "is", "great"]
model, vocab = train_fasttext(corpus, embedding_dim=50, epochs=5)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:01<00:00, 17.94it/s]


Epoch 1, Loss: 41.6985013759154


100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 214.86it/s]


Epoch 2, Loss: 24.534544253865114


100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 127.46it/s]


Epoch 3, Loss: 11.722990141462752


100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 159.51it/s]


Epoch 4, Loss: 2.8384052148978447


100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 129.02it/s]

Epoch 5, Loss: 6.967051905382994


In [16]:
model.word_embeddings.weight.data

tensor([[-0.9717,  0.6840,  0.0682,  ...,  0.4309, -1.5068,  1.2907],
        [-0.4200, -0.6563, -0.5629,  ...,  0.0742,  0.3027, -1.5285],
        [-0.2640,  0.6243, -0.0507,  ...,  1.1935,  0.2019,  0.2567],
        ...,
        [-0.8082, -0.2924, -0.6600,  ..., -0.3907,  1.1086,  0.1791],
        [ 1.2585,  2.0222,  2.1341,  ...,  1.0742, -1.9727,  1.9272],
        [-1.2519, -0.2590,  0.4494,  ..., -2.1394,  1.3334, -0.6800]])

In [17]:
model.subword_embeddings.weight.data

tensor([[-1.6084,  1.6022,  0.5933,  ...,  1.3943, -0.7877, -1.1504],
        [-0.3314,  0.1283,  0.4402,  ...,  0.2284, -0.0147,  0.3598],
        [-0.0056, -1.7762, -1.0277,  ...,  0.9704,  1.8002,  0.8024],
        ...,
        [-0.7988,  0.0888,  1.0798,  ..., -1.7981,  1.0652,  0.6942],
        [-1.4244,  1.4719,  1.3277,  ..., -1.0287,  0.3502,  0.2620],
        [-1.0979,  0.1336,  0.8745,  ..., -1.4876, -0.9884, -0.7860]])

In [18]:
len(list(vocab)),list(vocab)[:10],list(vocab)[-10:]

(147,
 ['I',
  'love',
  'natural',
  'language',
  'processing',
  'fasttext',
  'is',
  'great',
  'ttext',
  'text'],
 ['essin',
  '<lan',
  '<natur',
  'ces',
  '<fasttext>',
  '<love>',
  'asttex',
  'cess',
  'atur',
  'oces'])

--------